In [1]:
import os
from Bio import SeqIO
import io
from Bio.Seq import Seq
import subprocess

In [2]:
def make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length):
    for offset in range(3):
        translated_sequence = str(gene_sequence[offset:].translate()).split("*")
        for i in range(len(translated_sequence)):
            if len(translated_sequence[i])> 0.8*query_length:
                sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
                # print(i+1, offset)
                
                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:
                    output = f"{query_fasta_sequence}\n\n{sequence_set}"
                    out_file.write(output)

In [3]:
def run_mafft(annotated_genome_location,annotated_species_name,error_exon):
    location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment'
    list_of_files_to_run_mafft_on = os.listdir(location)
    for file in list_of_files_to_run_mafft_on:
        if file.endswith(".fa"):
            command = f'"mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
            # print(command)
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            # os.system(f'{command}')
            command = f'"mafft" --localpair --clustalout --maxiterate 16 --reorder "{location}/{file}" > "{location}/alignment_clustal_{file}.txt"'
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            
            # os.system(f'{command}')
    return(location)

In [4]:
def process_mafft_output(mafft_run_folder, error_exon):
    list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
    score_output = []
    score = 99
    min_score_sequence = ''
    alignment_file = ''
    for file in list_of_files_in_mafft_run_folder:
        if file.endswith(".fa.hat2"):
            with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
                dist_matrix_list = dist_matrix_file.readlines()
            
            sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
            distance_score = float(dist_matrix_list[-1].rstrip())
            if len(score_output) < 5:
                score_output.append( [sequence_name,distance_score])
            else:
                for i in range(len(score_output)):
                    score_at_this_index = score_output[i][1]
                    if distance_score < score_at_this_index:
                        score_output[i] = [sequence_name,distance_score]
                        break
            if distance_score < score:
                score = distance_score
                min_score_sequence = sequence_name
                alignment_file = f'alignment_{file.replace(".hat2",".txt")}'
    # print(score_output)
    print(f"min = {min_score_sequence}, {score}" )
    # score_out_merged = '\n'.join(score_output)
    print(f"5 top scores:\n{score_output}")
    print(f"{mafft_run_folder}/{alignment_file}")
    
    alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
    # print (records.id)
    fasta_start_position = 0
    fasta_end_position = 0
    start_switch = 0
    end_switch = 0
    alignment_name = ''
    for records in alignment_file:
        
        
        
        if error_exon in records.id:
            gap_counter = 0
            base_counter = 0
            for current_position in range(len(records.seq)):
                sequence_length = len(records.seq) - records.seq.count('-')
                # print(f"fasta_end_position {fasta_end_position} fasta_start_position {fasta_start_position}")
                # print(f"current_position = {current_position}, {len(records.seq)}")
                # print(records.seq[current_position])
                # print("fasta_start_position",fasta_start_position)
                # print("start_switch",start_switch)
                
                # print(gap_counter, base_counter)
                # print(5,0.2*sequence_length)
                if start_switch == 1 and records.seq[current_position] == "-" and gap_counter > 3 and (len(records.seq[:current_position]) - records.seq[:current_position].count('-')) < (0.1*sequence_length):
                #  :
                    # print("\n\nhere\n\n")
                    start_switch = 0
                    gap_counter = 0

                if "-" not in records.seq[current_position] and start_switch == 0 :
                    fasta_start_position = current_position
                    start_switch = 1
                
                    
                
                    
                if end_switch == 1 and "-" not in records.seq[current_position] and (len(records.seq[current_position:]) - records.seq[current_position:].count('-')) > 0.1*sequence_length :
                    end_switch = 0
                    gap_counter = 0
                
                    
                if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                    # print(f"base_counter {base_counter}")
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    fasta_end_position = current_position
                    end_switch = 1
                if "-" in records.seq[current_position]:
                    gap_counter += 1
                else:
                    gap_counter = 0
                    base_counter += 1
        else:
            alignment_name = records.id
            if fasta_end_position == 0:
                
                fasta_end_position = current_position
            end_switch = 1
            start_switch = 1

        if start_switch == 1 and end_switch == 1:
            print(fasta_start_position, fasta_end_position)
            gene_sequence = records.seq[fasta_start_position:fasta_end_position]
            print(f"{records.id}\n{gene_sequence}")
    return(gene_sequence, alignment_name)

# process_mafft_output(mafft_run_folder, error_exon)

In [5]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False

def check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon):
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    
    list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/")
    
    if error_exon not in list_of_exon_directories:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
    elif error_exon in list_of_exon_directories:
        list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        if "for_alignment" not in list_of_folders_1:
            os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{file}")


In [6]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
    print("Getting Gene")
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp")

    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    # subprocess.run("pwd")
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)

In [9]:
def mafft_process(previous_exon_coordinates,
                  next_exon_coordinates,
                  current_exon_coordinates,
                  query_species,
                  query_transcript,
                  query_location,
                  annotated_genome_location,
                 genome_location):
    query_exon = current_exon_coordinates[6].split("query")[-1][1:]
    error_exon = query_exon
    # print("AA")
    genome_file = get_genome_file(genome_location,species)
    
    upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    
    
    with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
        query_file_list = query_file.readlines()
        query_fasta_sequence = "".join(query_file_list)
        left_overhang = query_file_list[0].split("Frame")[1][1]
        right_overhang = query_file_list[0].split("rightoh")[1][1]
        original_query_name = query_file_list[0]
        original_query = query_file_list[1]
        
    query_length = len(query_fasta_sequence.split("\n")[1])
    
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)
    # print(annotated_species_name)

    # print(annotated_genome_location)
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    # assert False
    check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon)
    
    
    make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length)
    
    return(annotated_species_name, error_exon,species, genome_file,gene_sequence,left_overhang,right_overhang,scaffold, original_query_name,original_query )


In [11]:

blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result"
# blast_output_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
species_list = os.listdir(blast_output_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
for species in species_list:
    annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result"
    query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Query"
    list_of_query_species = os.listdir(query_location)
    list_of_query_species.remove("desktop.ini")
    for query_species in list_of_query_species:
        list_of_query_transcripts = os.listdir(f"{query_location}/{query_species}")
        list_of_query_transcripts.remove("desktop.ini")
        for query_transcript in list_of_query_transcripts:


            genome_location = "/mnt/f/Genomes_2023-12-26"



            output_coordinate_file = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"
            exons_to_check_list = ["Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15","Exon_16","Exon_17","Exon_18","Exon_19","Exon_20","Exon_21","Exon_22","Exon_23","Exon_24","Exon_25"]
#             exons_to_check_list = ["Exon_21"]

            list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
            coordinate_file_name = ''
        #     print(list_of_files_in_species_folder)
            for file_names in list_of_files_in_species_folder:

                if file_names.endswith("_coordinates_old.csv"):
                    coordinate_file_name = file_names
            if coordinate_file_name =='':
                print(f"Coordinate file error")
                assert False

            with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
                coordinate_file_lines = temp_file_open.readlines()
            # print(coordinate_file_lines[0])
            for exons_to_check in exons_to_check_list:

                for i in range(1,len(coordinate_file_lines)):
                    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
                    current_exon_coordinates = coordinate_file_lines[i].split(",")
                    if i == 1:
                        scaffold_first_exon = coordinate_file_lines[i].split(",")[1]
                        first_exon_start, first_exon_end = coordinate_file_lines[i].split(",")[2],coordinate_file_lines[i].split(",")[3]
                    # if coordinate_file_lines_split[5] == "Y":
                    if  coordinate_file_lines_split[6].endswith(exons_to_check):
                        if i == 1:
                            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
                            assert False
                        else:
                            print(f"Scaffold = {scaffold_first_exon}, start = {first_exon_start}, end = {first_exon_end}")
                            print(coordinate_file_lines[i])
                            # process_current_exon = input("Error Found! Process?")
                            process_current_exon = 'y'
                            if process_current_exon.lower()[0] == "y":
                                if i>2:
                                    previous_exon_number = i-2
                                else:
                                    previous_exon_number = i-1
                                if i+2 <=int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = i+2
                                else:
                                    next_exon_number = i+1
                                if i >= int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = int(exons_to_check_list[-1].split("_")[1])
#                                     continue

                                '''
                                the above code ensures that missing exons do not cause issues
                                '''

                                previous_exon_coordinates = ''
                                next_exon_coordinates = ''
                                while True:
                                    print(previous_exon_number)
                                    if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                                        previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                        break
                                    # else:
                                    #     proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                                    #     if proceed_prompt.lower()[0] == "y":
                                    #         previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                    #         break
                                    previous_exon_number -= 1
                                while True:
                                    print(next_exon_number)
                                    try:
                                        if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number]
                                            break
                                    except:
                                        if coordinate_file_lines[next_exon_number-1].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number-1]
                                            break
                                   
                                    next_exon_number += 1


                                print(previous_exon_coordinates)
                                print(next_exon_coordinates)
                                annotated_species_name, error_exon, species, genome_file, gene_sequence,left_overhang,right_overhang,scaffold,original_query_name, original_query = mafft_process(previous_exon_coordinates,
                                                                                                                                  next_exon_coordinates,
                                                                                                                                  current_exon_coordinates,
                                                                                                                                  query_species,
                                                                                                                                  query_transcript,
                                                                                                                                  query_location,
                                                                                                                                  annotated_genome_location,
                                                                                                                                     genome_location)

                                mafft_run_folder = run_mafft(annotated_genome_location,annotated_species_name,error_exon)

                                possible_gene_sequence, alignment_name = process_mafft_output(mafft_run_folder, error_exon)

                                print(f"Original query = {original_query}")
                                print(f"New query = {possible_gene_sequence}")
                                
                                list_of_folders_inside_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
                                if "for_blast" not in list_of_folders_inside_exon_folder:
                                    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                else:
                                    list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                    for file in list_of_files:
                                        os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/{file}")


                                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/new_query.txt",'w') as query_file:
                                    query = f">Query_{error_exon}_{alignment_name}\n{possible_gene_sequence}"
                                    query_file.write(query)
                    

Scaffold = NC_059568.1, start = 12338695, end = 12338780
Colias_croceus,NC_059568.1,12340826,12340909,0,N,32.Colias_croceus_XM_045653731.1_Frame_1_rightoh_2_query_Exon_2,1,27,27

1
4
Colias_croceus,NC_059568.1,12338695,12338780,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_2_query_Exon_1,1,28,28

Colias_croceus,NC_059568.1,12346414,12346502,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_0_query_Exon_4,1,29,29

Getting Gene
min = set45_frame2, 0.143
5 top scores:
[['set45_frame2', 0.143], ['set13_frame0', 1.363], ['set17_frame2', 1.54], ['set18_frame0', 1.587], ['set70_frame2', 1.63]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Colias_croceus/Period_gene_genomic_sequence_individual_exon_28.Maniola_jurtina/Exon_2/for_alignment/alignment_Exon_2_translated_genomic_sequence_45_frame2.fa.txt
5 32
28.Maniola_jurtina_XM_045928854.1_Frame_1_rightoh_2_query_Exon_2
RSSKSTHSGSNSSGSSGYGGKPSTSGY
5 32
set45_frame2
RSSHSTHSGSHSSGSSGYGG

min = set53_frame2, 0.524
5 top scores:
[['set24_frame0', 1.889], ['set48_frame2', 1.833], ['set53_frame2', 0.524], ['set73_frame2', 1.885]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Colias_croceus/Period_gene_genomic_sequence_individual_exon_28.Maniola_jurtina/Exon_9/for_alignment/alignment_Exon_9_translated_genomic_sequence_53_frame2.fa.txt
20 95
28.Maniola_jurtina_XM_045928854.1_Frame_2_rightoh_2_query_Exon_9
PNEIVTNPKPFVMRHSPSGRLDYIDPESVPYLGYLPQDIIEKDVLLLYHPSDLFYLRQVYETIVKEGGEQRSKPY
20 95
set53_frame2
PNELVSKAVPFVIRHAANGDIKYIDPESVPYLGYSPQDIVNKDALHLYHPHDLAYLRQVYETIVKEGGVPRSKPY
Original query = PNEIVTNPKPFVMRHSPSGRLDYIDPESVPYLGYLPQDIIEKDVLLLYHPSDLFYLRQVYETIVKEGGEQRSKPY
New query = PNELVSKAVPFVIRHAANGDIKYIDPESVPYLGYSPQDIVNKDALHLYHPHDLAYLRQVYETIVKEGGVPRSKPY
Scaffold = NC_059568.1, start = 12338695, end = 12338780
Colias_croceus,NC_059568.1,12353151,12353262,0,N,32.Colias_croceus_XM_045653731.1_Frame_1_rightoh_0_query_Exon_10,1,37,

min = set51_frame1, 0.642
5 top scores:
[['set64_frame2', 1.699], ['set76_frame0', 1.851], ['set21_frame1', 1.856], ['set51_frame1', 0.642], ['set76_frame1', 1.825]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Colias_croceus/Period_gene_genomic_sequence_individual_exon_28.Maniola_jurtina/Exon_16/for_alignment/alignment_Exon_16_translated_genomic_sequence_51_frame1.fa.txt
15 70
28.Maniola_jurtina_XM_045928854.1_Frame_1_rightoh_0_query_Exon_16
HNIDMEKELLKKHRETRSSSKGEREKASNESRQKKKEHLARCNALYQPTTAGLSA
15 70
set51_frame1
HNAVMEKELMKLHREIKTVTKDDRKKASKESRQKKKEHLARCNASFLPTSAGLST
Original query = HNIDMEKELLKKHRETRSSSKGEREKASNESRQKKKEHLARCNALYQPTTAGLSADK
New query = HNAVMEKELMKLHREIKTVTKDDRKKASKESRQKKKEHLARCNASFLPTSAGLST
Scaffold = NC_059568.1, start = 12338695, end = 12338780
Colias_croceus,NC_059568.1,12357405,12357552,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_1_query_Exon_17,1,49,49

15
19
Colias_croceus,NC_059568.1,12355091,1235

Getting Gene
min = set113_frame2, 0.505
5 top scores:
[['set113_frame2', 0.505], ['set120_frame0', 1.675], ['set107_frame2', 1.681], ['set70_frame1', 1.717], ['set15_frame2', 1.719]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Colias_croceus/Period_gene_genomic_sequence_individual_exon_28.Maniola_jurtina/Exon_24/for_alignment/alignment_Exon_24_translated_genomic_sequence_113_frame2.fa.txt
3 37
28.Maniola_jurtina_XM_045928854.1_Frame_0_rightoh_0_query_Exon_24
VCVTSELIYKYQIVTKNIDEVLSSDKEKLKQLEQ
3 37
set113_frame2
VCVTSELIYKYQVLPKTMEEVLSCDKKRLETLEQ
Original query = VCVTSELIYKYQIVTKNIDEVLSSDKEKLKQLEQ
New query = VCVTSELIYKYQVLPKTMEEVLSCDKKRLETLEQ
Scaffold = NC_059568.1, start = 12338695, end = 12338780
Colias_croceus,NC_059568.1,12371299,12371442,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_25,1,48,48

23
25
Colias_croceus,NC_059568.1,12370479,12370565,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exo

31 56
set57_frame0
DRNTFASLITNGLAAPKTVNGTQKK
Original query = DRNTFASQITNGVAIPKIVHRTHHK
New query = DRNTFASLITNGLAAPKTVNGTQKK
Scaffold = NC_059568.1, start = 12338695, end = 12338780
Colias_croceus,NC_059568.1,12351686,12351877,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_1_query_Exon_8,1,63,63

6
10
Colias_croceus,NC_059568.1,12348126,12348257,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_6,1,44,44

Colias_croceus,NC_059568.1,12353151,12353262,0,N,32.Colias_croceus_XM_045653731.1_Frame_1_rightoh_0_query_Exon_10,1,37,37

Getting Gene
min = set81_frame1, 0.453
5 top scores:
[['set109_frame0', 1.805], ['set86_frame1', 1.813], ['set31_frame1', 1.839], ['set106_frame1', 1.839], ['set81_frame1', 0.453]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Colias_croceus/Period_gene_genomic_sequence_individual_exon_25.Pieris_napi/Exon_8/for_alignment/alignment_Exon_8_translated_genomic_sequence_81_frame1.fa.txt
0 72
25.

...

min = set32_frame0, 0.255
5 top scores:
[['set32_frame0', 0.255], ['set53_frame2', 1.7], ['set58_frame1', 1.782], ['set58_frame2', 1.786], ['set28_frame0', 1.814]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Zerene_cesonia/Period_gene_genomic_sequence_individual_exon_32.Colias_croceus/Exon_11/for_alignment/alignment_Exon_11_translated_genomic_sequence_32_frame0.fa.txt
0 40
32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_11
GPNNPDVFQSADIEKSFKFSEDEKNKAQTLRESIVRIMNE
0 40
set32_frame0
GPSNPDVFQSADMDKTIKFSEEEKNKAQSLRESIVRIMNE
Original query = GPNNPDVFQSADIEKSFKFSEDEKNKAQTLRESIVRIMNE
New query = GPSNPDVFQSADMDKTIKFSEEEKNKAQSLRESIVRIMNE
Scaffold = NC_052122.1, start = 2287111, end = 2287196
Zerene_cesonia,NC_052122.1,2298651,2298800,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_12,1,50,50

10
14
Zerene_cesonia,NC_052122.1,2297650,2297761,0,N,32.Colias_croceus_XM_045653731.1_Frame_1_rightoh_0_query_Exon_10,

min = set20_frame2, 0.978
5 top scores:
[['set20_frame2', 0.978], ['set23_frame1', 1.691], ['set37_frame1', 1.737], ['set11_frame0', 1.697], ['set44_frame2', 1.76]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Zerene_cesonia/Period_gene_genomic_sequence_individual_exon_32.Colias_croceus/Exon_19/for_alignment/alignment_Exon_19_translated_genomic_sequence_20_frame2.fa.txt
12 47
32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_1_query_Exon_19
MFPICYAMPASSQQMMQNDTEMPNQGPSTSNHPFT
12 47
set20_frame2
MFPICY-VPATSQHTTHQPG-MANQEPTTSNHPFT
Original query = MFPICYAMPASSQQMMQNDTEMPNQGPSTSNHPFT
New query = MFPICY-VPATSQHTTHQPG-MANQEPTTSNHPFT
Scaffold = NC_052122.1, start = 2287111, end = 2287196
Zerene_cesonia,NC_052122.1,2302800,2302957,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_0_query_Exon_20,1,54,54

18
22
Zerene_cesonia,NC_052122.1,2301924,2302031,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_1_query_Exon_18,1,37,37

Zerene_c

Scaffold = NC_052122.1, start = 2287111, end = 2287196
Zerene_cesonia,NC_052122.1,2290289,2290443,0,N,32.Colias_croceus_XM_045653731.1_Frame_1_rightoh_1_query_Exon_3,1,51,51

1
5
Zerene_cesonia,NC_052122.1,2287111,2287196,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_2_query_Exon_1,1,28,28

Zerene_cesonia,NC_052122.1,2291659,2291715,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_5,1,19,19

Getting Gene
min = set74_frame2, 0.0
5 top scores:
[['set74_frame2', 0.0], ['set37_frame1', 1.793], ['set106_frame1', 1.816], ['set60_frame2', 1.842], ['set76_frame0', 1.701]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Zerene_cesonia/Period_gene_genomic_sequence_individual_exon_14.Zerene_cesonia/Exon_3/for_alignment/alignment_Exon_3_translated_genomic_sequence_74_frame2.fa.txt
15 66
14.Zerene_cesonia_XM_038360763.1_Frame_1_rightoh_1_query_Exon_3
SNNVSQPPVEIRCKVKKKKKQPEETLEVQEEEPSPPEPEPEVKPVEEECVD
15 66
set74_frame2
SNNVS

min = set37_frame0, 0.0
5 top scores:
[['set37_frame0', 0.0], ['set1_frame2', 1.691], ['set4_frame2', 1.725], ['set24_frame2', 1.796], ['set44_frame1', 1.799]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Zerene_cesonia/Period_gene_genomic_sequence_individual_exon_14.Zerene_cesonia/Exon_10/for_alignment/alignment_Exon_10_translated_genomic_sequence_37_frame0.fa.txt
18 55
14.Zerene_cesonia_XM_038360763.1_Frame_1_rightoh_0_query_Exon_10
MMAQNGDYVKLETEWTSFINPWSRKLEFVIGKHLLIE
18 55
set37_frame0
MMAQNGDYVKLETEWTSFINPWSRKLEFVIGKHLLIE
Original query = MMAQNGDYVKLETEWTSFINPWSRKLEFVIGKHLLIE
New query = MMAQNGDYVKLETEWTSFINPWSRKLEFVIGKHLLIE
Scaffold = NC_052122.1, start = 2287111, end = 2287196
Zerene_cesonia,NC_052122.1,2297933,2298052,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_0_query_Exon_11,1,40,40

9
13
Zerene_cesonia,NC_052122.1,2296400,2296628,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_2_query_Exon_9,1,75,75

Zerene_

min = set19_frame1, 0.0
5 top scores:
[['set19_frame1', 0.0], ['set5_frame0', 1.578], ['set10_frame2', 1.681], ['set32_frame2', 1.735], ['set20_frame0', 1.756]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/5.Coliadinae/1.Blast_result/Zerene_cesonia/Period_gene_genomic_sequence_individual_exon_14.Zerene_cesonia/Exon_18/for_alignment/alignment_Exon_18_translated_genomic_sequence_19_frame1.fa.txt
9 44
14.Zerene_cesonia_XM_038360763.1_Frame_2_rightoh_1_query_Exon_18
STPMPTNSMNNMNAFFLGVSIPQQMSLMSPVAVMP
9 44
set19_frame1
STPMPTNSMNNMNAFFLGVSIPQQMSLMSPVAVMP
Original query = STPMPTNSMNNMNAFFLGVSIPQQMSLMSPVAVMP
New query = STPMPTNSMNNMNAFFLGVSIPQQMSLMSPVAVMP
Scaffold = NC_052122.1, start = 2287111, end = 2287196
Zerene_cesonia,NC_052122.1,2302384,2302485,0,N,32.Colias_croceus_XM_045653731.1_Frame_2_rightoh_1_query_Exon_19,1,35,35

17
21
Zerene_cesonia,NC_052122.1,2301661,2301808,0,N,32.Colias_croceus_XM_045653731.1_Frame_0_rightoh_1_query_Exon_17,1,49,49

Zerene_cesonia